In [1]:
!pip install opencv-python face_recognition scikit-learn

#Capture Face Images from Camera: Capture images of faces and save them in separate directories for each person.

In [2]:
import cv2
import os

# Create directory to save face images
def create_dir(nama):
    os.makedirs(f'dataset/{nama}', exist_ok=True)

# Capture images from webcam
def capture_images(nama, max_photos=100):
    create_dir(nama)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open the webcam.")
        return
    
    # Optionally set a smaller resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    count = 0
    saved_images = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Show the live frame to the user
        cv2.imshow('Capture Face', frame)

        # Save every 5th frame to ensure enough samples
        if count % 5 == 0 and saved_images < max_photos:
            face_path = f'dataset/{nama}/{nama}_{saved_images}.jpg'
            cv2.imwrite(face_path, frame)
            saved_images += 1

        count += 1

        # Stop if 500 images are captured
        if saved_images >= max_photos:
            print(f"Captured {max_photos} images.")
            break

        # Press 'q' to exit capturing early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example of how to call the function to capture images for a person
nama = input("Enter the name of the person: ")
capture_images(nama)

Captured 100 images.


#Preprocessing and Face Detection: After capturing images, we need to detect faces in each image, extract them, and save the cropped face.

In [5]:
import cv2
import face_recognition
import os

# Detect and crop faces from the images in the dataset
def process_images(base_folder='dataset'):
    # Loop through each folder in the base directory
    for nama in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, nama)
        
        # Ensure the path is a directory
        if os.path.isdir(folder_path):
            for img_name in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_name)
                image = cv2.imread(img_path)

                # Check if the image was loaded properly
                if image is not None:
                    face_locations = face_recognition.face_locations(image)

                    for i, (top, right, bottom, left) in enumerate(face_locations):
                        face_image = image[top:bottom, left:right]
                        face_path = os.path.join(folder_path, f'face_{i}_{img_name}')
                        cv2.imwrite(face_path, face_image)
                else:
                    print(f"Warning: Could not read image {img_path}")

# Call the function to process images in all folders
process_images()

# Example usage:
# process_images(nama)

In [6]:
#Extract Face Embeddings: Face embeddings convert each face into a fixed-size vector. You can use face_recognition for generating these embeddings.

In [7]:
import numpy as np
import face_recognition
import os

def get_face_encodings(nama):
    encodings = []
    labels = []
    folder_path = f'dataset/{nama}/'

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        image = cv2.imread(img_path)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get face encodings
        face_encodings = face_recognition.face_encodings(rgb_image)

        for encoding in face_encodings:
            encodings.append(encoding)
            labels.append(nama)

    return encodings, labels

# Collect data for all people
all_encodings = []
all_labels = []

for person_name in os.listdir('dataset'):
    encodings, labels = get_face_encodings(person_name)
    all_encodings.extend(encodings)
    all_labels.extend(labels)

X = np.array(all_encodings)
y = np.array(all_labels)


In [8]:
import numpy as np

# Misalkan all_encodings adalah daftar encoding wajah dan all_labels adalah daftar nama
known_face_encodings = all_encodings  # Encoding wajah yang kamu hasilkan
known_face_names = all_labels  # Nama-nama dari orang yang dikenali

# Simpan encoding wajah dan nama menggunakan np.save
np.save('known_face_encodings.npy', known_face_encodings)
np.save('known_face_names.npy', known_face_names)


#Train a Classifier (SVM or k-NN): Once the embeddings are collected, you can train a machine learning model like SVM or k-NN to classify the faces based on the embeddings.

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an SVM classifier
classifier = SVC(kernel='linear', probability=True)
classifier.fit(X_train, y_train)

# Test the classifier
y_pred = classifier.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

Accuracy: 0.9809523809523809


In [10]:
import joblib

# Simpan model ke dalam file
joblib.dump(classifier, 'svm_face_recognition_model.pkl')
print("Model has been saved.")

Model has been saved.


#Real-Time Face Recognition: Once the model is trained, use it to recognize faces from the webcam in real-time.

In [11]:
import cv2
import face_recognition

def recognize_faces(classifier):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB (face_recognition uses RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face locations and encodings in the frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Iterate over each detected face and recognize it
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Predict the person
            probabilities = classifier.predict_proba([face_encoding])[0]
            best_match = np.argmax(probabilities)
            name = classifier.classes_[best_match]
            confidence = probabilities[best_match]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Label the face
            label = f'{name} ({confidence:.2f})'
            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Display the resulting image
        cv2.imshow('Face Recognition', frame)

        # Break the loop on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Recognize faces using the trained SVM classifier
recognize_faces(classifier)

KeyboardInterrupt: 

: 

# Summary:
- Data Collection: Capture face images using OpenCV.
- Face Detection: Detect and crop faces using face_recognition.
- Face Embeddings: Extract face embeddings (feature vectors) from the face images.
- Model Training: Train a classifier (SVM or k-NN) using the face embeddings.
- Real-Time Recognition: Use the trained classifier for real-time face recognition from the camera.